## SpaceX contracts and subcontracts

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.float_format", lambda x: "%.2f" % x)

### read in data

In [26]:
spacex_duns = "120406462"

In [6]:
spacex_prime = pd.read_csv(
    "data/raw/Contracts_PrimeAwardSummaries_2022-04-27_H17M36S48_1.csv",
    dtype={"recipient_parent_duns": str},
)

In [7]:
spacex_prime.total_obligated_amount.sum()

10695208656.380001

In [8]:
spacex_prime_df = (
    spacex_prime[spacex_prime["recipient_parent_duns"] == spacex_duns][
        [
            "award_id_piid",
            "recipient_parent_name",
            "award_description",
            "awarding_agency_name",
            "awarding_sub_agency_name",
            "type_of_contract_pricing",
            "total_obligated_amount",
            "potential_total_value_of_award",
            "award_base_action_date",
            "award_base_action_date_fiscal_year",
        ]
    ]
    .sort_values("total_obligated_amount", ascending=False)
    .copy()
)

In [9]:
spacex_prime_df.total_obligated_amount.sum().round()

7327613208.0

In [10]:
spacex_prime_df.head()

,award_id_piid,recipient_parent_name,award_description,awarding_agency_name,awarding_sub_agency_name,type_of_contract_pricing,total_obligated_amount,potential_total_value_of_award,award_base_action_date,award_base_action_date_fiscal_year
66,NNJ09GA04B,SPACE EXPLORATION TECHNOLOGIES CORP.,ISS COMMERCIAL RESUPPLY SERVICES.,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION ...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,FIRM FIXED PRICE,3011395263.58,3100000000.00,2008-12-22,2009
21,NNK14MA74C,SPACE EXPLORATION TECHNOLOGIES CORP.,IGF::OT::IGF THE COMMERCIAL CREW PROGRAM (CCP...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION ...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,FIRM FIXED PRICE,1240950821.00,3511811270.00,2014-09-16,2014
64,80MSFC20C0034,SPACE EXPLORATION TECHNOLOGIES CORP.,"WORK REQUIRED FOR THE DESIGN, DEVELOPMENT, MAN...",NATIONAL AERONAUTICS AND SPACE ADMINISTRATION ...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,FIRM FIXED PRICE,530667416.24,3031455921.24,2020-05-13,2020
63,FA881119C0004,SPACE EXPLORATION TECHNOLOGIES CORP.,EVOLVED EXPENDABLE LAUNCH VEHICLE 1A-6 LAUNCH ...,DEPARTMENT OF DEFENSE (DOD),DEPT OF THE AIR FORCE,FIRM FIXED PRICE,310871630.30,310871630.30,2019-02-21,2019
61,FA881118C0001,SPACE EXPLORATION TECHNOLOGIES CORP.,IGF::OT::IGF EVOLVED EXPENDABLE LAUNCH VEHICLE...,DEPARTMENT OF DEFENSE (DOD),DEPT OF THE AIR FORCE,FIRM FIXED PRICE,243895249.55,437551474.55,2018-03-14,2018


### read in subawards

In [27]:
spacex_subs = pd.read_csv(
    "data/raw/Contracts_Subawards_2022-04-27_H17M36S54_1.csv",
    dtype={"subawardee_parent_duns": str},
)

In [54]:
spacex_subs_df = spacex_subs[spacex_subs["subawardee_parent_duns"] == spacex_duns][
    [
        "subaward_number",
        "subaward_type",
        "subaward_amount",
        "subawardee_parent_duns",
        "subawardee_parent_name",
        "subawardee_name",
        "subaward_action_date",
        "subaward_action_date_fiscal_year",
        "prime_award_awarding_sub_agency_name",
        "prime_awardee_parent_name",
        "prime_awardee_parent_duns",
    ]
].drop_duplicates(subset="subaward_number")

#### Companies

In [43]:
companies = (
    spacex_subs_df.groupby(["prime_awardee_parent_name"])
    .agg({"subaward_amount": sum})
    .reset_index()
    .sort_values("subaward_amount", ascending=False)
)
companies["share"] = (
    companies["subaward_amount"] / companies["subaward_amount"].sum()
) * 100
companies

,prime_awardee_parent_name,subaward_amount,share
3,NORTHROP GRUMMAN CORPORATION,10756557.00,62.38
1,"L3HARRIS TECHNOLOGIES, INC.",4166879.43,24.17
0,"L3 TECHNOLOGIES, INC.",800000.00,4.64
4,PERSPECTA INC.,794223.00,4.61
2,Maxar Technologies Inc.,724961.99,4.20


#### Agencies

In [41]:
subs_agencies = (
    spacex_subs_df.groupby(["prime_award_awarding_sub_agency_name"])
    .agg({"subaward_amount": sum})
    .reset_index()
    .sort_values("subaward_amount", ascending=False)
)
subs_agencies["share"] = (
    subs_agencies["subaward_amount"] / companies["subaward_amount"].sum()
) * 100
subs_agencies = subs_agencies[subs_agencies["subaward_amount"] > 0]
subs_agencies

,prime_award_awarding_sub_agency_name,subaward_amount,share
1,DEPT OF THE AIR FORCE,16448398.42,95.39
2,SPACE DEVELOPMENT AGENCY,700000.00,4.06
0,DEFENSE ADVANCED RESEARCH PROJECTS AGENCY (DA...,94223.00,0.55


In [47]:
spacex_subs_total = (
    spacex_subs_df.groupby(["subawardee_name"])
    .agg({"subaward_amount": sum})
    .reset_index()
)

#### Years

In [48]:
years = (
    spacex_subs_df.groupby(["subaward_action_date_fiscal_year"])
    .agg({"subaward_amount": sum})
    .reset_index()
    .sort_values("subaward_action_date_fiscal_year", ascending=False)
)
years["share"] = (
    years["subaward_amount"] / spacex_subs_total["subaward_amount"].sum()
) * 100
years = years[years["subaward_amount"] > 0]
years

,subaward_action_date_fiscal_year,subaward_amount,share
2,2021,4866879.43,28.23
1,2020,11575741.99,67.13
0,2019,800000.00,4.64


---

## Export

In [42]:
subs_agencies.to_csv("data/processed/spacex_subcontract_agencies.csv")

In [44]:
companies.to_csv("data/processed/spacex_subcontract_companies.csv")

In [45]:
spacex_subs_df.to_csv("data/processed/spacex_subcontract_allcontracts.csv")

In [49]:
years.to_csv("data/processed/spacex_subcontract_years.csv")